In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.window import Window
from pyspark.sql.functions import current_timestamp, lit
from pyspark.sql.utils import AnalysisException
from datetime import datetime


In [0]:
src_silver_path="/Volumes/customer_360/customer_360_silver/silver_product_volume"
gold_path="/Volumes/customer_360/customer_360_gold/gold_product_dim_volume"

In [0]:
spark=SparkSession.builder.appName("read silver data").getOrCreate()
df=spark.read\
    .format("delta")\
    .load(src_silver_path)

In [0]:
if DeltaTable.isDeltaTable(spark, gold_path):
    bronze_table = DeltaTable.forPath(spark, gold_path)
    # Get max data_arrival_timestamp
    max_ts_row = bronze_table.toDF().select(max("data_arrival_timestamp")).collect()[0]
    max_ts = max_ts_row[0]  # None if table is empty
    if max_ts is None:
        print("e table is empty. Will load all records.")
else:
    print(" table not found. Will load all records.")
    max_ts = None  # first load

# Filter source for incremental load
if max_ts:
    df = df.filter(col("data_arrival_timestamp") > max_ts)
else:
    df = df # first load, take all records

print(f"Number of records to load: {df.count()}")

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp
from pyspark.sql.types import TimestampType
from delta.tables import DeltaTable


# --- Step 1: Add tracking columns ---
# Assuming df is your product DataFrame
df = df.withColumn("start_date", current_timestamp()) \
                       .withColumn("end_date", lit(None).cast(TimestampType())) \
                       .withColumn("is_active", ~col("is_deleted"))  # active if not deleted

df_written_product = None

# --- Step 2: Check if Delta table exists ---
if DeltaTable.isDeltaTable(spark, gold_path):
    print("✅ Product table exists. Applying SCD Type 2 logic...")

    delta_table = DeltaTable.forPath(spark, gold_path)
    df_existing = delta_table.toDF()
    df_active_existing = df_existing.filter(col("is_active") == True)

    # --- Step 3: Detect changed products ---
    changes_df = df.alias("src").join(
        df_active_existing.alias("tgt"),
        on=col("src.product_id") == col("tgt.product_id"),
        how="inner"
    ).filter(
        (col("src.category") != col("tgt.category")) |
        (col("src.product_name") != col("tgt.product_name")) |
        (col("src.is_deleted") != col("tgt.is_deleted"))
    ).select("src.product_id").distinct()

    # --- Step 4: Detect new products ---
    df_new_products = df.alias("src").join(
        df_active_existing.alias("tgt"),
        on=col("src.product_id") == col("tgt.product_id"),
        how="left_anti"
    )

    # --- Step 5: Update changed products ---
    if changes_df.count() > 0:
        print("🔁 Found changed products — applying SCD Type 2 updates...")
        changed_ids = [r["product_id"] for r in changes_df.collect()]

        delta_table.update(
            condition=col("product_id").isin(changed_ids) & (col("is_active") == True),
            set={
                "is_active": lit(False),
                "end_date": current_timestamp()
            }
        )

        df_changed_products = df.join(changes_df, on="product_id", how="inner")
    else:
        df_changed_products = spark.createDataFrame([], df.schema)

    # --- Step 6: Combine new + changed products ---
    df_to_insert = df_new_products.unionByName(df_changed_products)

    # --- Step 7: Write if there are updates ---
    if df_to_insert.count() > 0:
        print(f"📥 Appending {df_to_insert.count()} new/changed products...")
        df_written = spark.createDataFrame(df_to_insert.collect(), schema=df_to_insert.schema)
        df_to_insert.write \
            .format("delta") \
            .mode("append") \
            .option("mergeSchema", "true") \
            .partitionBy("category") \
            .save(gold_path)
        print("✅ SCD Type 2 update completed successfully.")
    else:
        print("ℹ️ No new or changed products. Nothing to insert.")
        df_written = spark.createDataFrame([], df.schema)

else:
    print("🚀 Creating new Product Delta table (first load)...")
    df_written = spark.createDataFrame(df.collect(), schema=df.schema)
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .partitionBy("category") \
        .save(gold_path)
    print("✅ New Product Delta table created.")

# df_written_product is for audit purpose


In [0]:
df_written.display()

In [0]:

# Count records
records_count = df_written.count()

if records_count==0:
    msg=" (Source Empty)"
else:
    msg=""

# max timestamp (only if rows exist)
max_data_ts_row = (
    df_written.select(max("data_arrival_timestamp")).collect()[0][0]
    if records_count > 0
    else None
)

# Use Python datetime for load_time
load_time = datetime.now()

# Define schema explicitly
schema = StructType([
    StructField("layer", StringType(), True),
    StructField("table_name", StringType(), True),
    StructField("load_time", TimestampType(), True),
    StructField("records_loaded", LongType(), True),
    StructField("max_data_timestamp", TimestampType(), True)
])

# Prepare audit data (even if 0 rows)
data = [("gold", f"gold_product{msg}", load_time, records_count, max_data_ts_row)]

# Create DataFrame
df_audit = spark.createDataFrame(data, schema)

# Append to audit table
df_audit.write.format("delta") \
    .mode("append") \
    .save("/Volumes/customer_360/audit/audit_volume/etl_audit")

print(f"Audit log updated successfully. Records loaded: {records_count}")


In [0]:
%sql
select * from delta.`/Volumes/customer_360/customer_360_gold/gold_product_dim_volume`

In [0]:
%sql
-- TRUNCATE TABLE delta.`/Volumes/customer_360/customer_360_gold/gold_product_dim_volume`

In [0]:
%sql
-- UPDATE delta.`/Volumes/customer_360/customer_360_gold/gold_product_dim_volume`
-- SET product_name = 'gg_18'
-- WHERE product_id = 'PROD0001';
